In [1]:
import numpy as np 
import pandas as pd
from pandas_datareader import data as web 
import matplotlib.pyplot as plt 
import matplotlib.dates as mdates 
%matplotlib inline

import datetime as dt 
import mplfinance as mpf 

import time

import os
from os import listdir
from os.path import isfile, join

import statsmodels.api as sm
import seaborn as sns
from statsmodels.tsa.ar_model import AutoReg, ar_select_order

In [2]:
PATH = "C:\\Users\\nafissaad\\Finance\\Stock List\\"

# Start date defaults
S_YEAR = 2017
S_MONTH = 1
S_DAY = 3
S_DATE_STR = "2017-01-03"
S_DATE_DATETIME = dt.datetime(S_YEAR, S_MONTH, S_DAY)

# End date defaults
E_YEAR = 2021
E_MONTH = 8
E_DAY = 19
E_DATE_STR = "2021-08-19"
E_DATE_DATETIME = dt.datetime(E_YEAR, E_MONTH, E_DAY)

risk_free_rate = 0.0125 # Approximate 10 year bond rate

In [3]:
def save_dataframe_to_csv(df, ticker):
    df.to_csv(PATH + ticker + '.csv')

In [4]:
def get_df_from_csv(ticker):
    try:
        df = pd.read_csv(PATH + ticker + '.csv', index_col='Date', 
                         parse_dates=True)
    except FileNotFoundError:
        pass
        # print("File Doesn't Exist")
    else:
        return df

In [5]:
def add_daily_return_to_df(df, ticker):
    df['daily_return'] = (df['Adj Close'] / df['Adj Close'].shift(1)) - 1
    save_dataframe_to_csv(df, ticker)
    return df 

In [6]:
def merge_df_by_column_name(col_name, sdate, edate, *tickers):
    # Will hold data for all dataframes with the same column name
    mult_df = pd.DataFrame()
    
    for x in tickers:
        df = get_df_from_csv(x)
        
        # NEW Check if your dataframe has duplicate indexes
        if not df.index.is_unique:
            # Delete duplicates 
            df = df.loc[~df.index.duplicated(), :]
        
        mask = (df.index >= sdate) & (df.index <= edate)
        mult_df[x] = df.loc[mask][col_name]
        
    return mult_df

In [7]:
port_list = ["AMD", "CPRT"]
mult_df = merge_df_by_column_name('daily_return',  '2018-01-02', 
                                  '2021-09-10', *port_list)
mult_df

,AMD,CPRT
Date,,
2018-01-02,0.068093,0.009493
2018-01-03,0.051913,-0.004816
2018-01-04,0.049351,0.008066
2018-01-05,-0.019802,-0.004801
2018-01-08,0.033670,0.000459
...,...,...
2021-09-03,0.006593,0.003929
2021-09-07,-0.007005,-0.018059
2021-09-08,-0.027302,0.011678


In [8]:
#get S&P and AMD Data
sp_df = web.DataReader('^GSPC','yahoo','2017-1-3','2021-9-10')['Adj Close']
amd_df = get_df_from_csv('AMD')
amd_df

,Adj Close,daily_return
Date,,
2017-01-03,11.430000,NaN
2017-01-04,11.430000,0.000000
2017-01-05,11.240000,-0.016623
2017-01-06,11.320000,0.007117
2017-01-09,11.490000,0.015018
...,...,...
2021-09-03,109.919998,0.006593
2021-09-07,109.150002,-0.007005
2021-09-08,106.169998,-0.027302


In [9]:
save_dataframe_to_csv(sp_df,'^GSPC')
sp_df2 = get_df_from_csv('^GSPC')
add_daily_return_to_df(sp_df2,'^GSPC')

,Adj Close,daily_return
Date,,
2017-01-03,2257.830078,NaN
2017-01-04,2270.750000,0.005722
2017-01-05,2269.000000,-0.000771
2017-01-06,2276.979980,0.003517
2017-01-09,2268.899902,-0.003549
...,...,...
2021-09-03,4535.430176,-0.000335
2021-09-07,4520.029785,-0.003396
2021-09-08,4514.069824,-0.001319


In [12]:
#find Beta Stock Versus S&P
def find_beta(ticker):
    port_list = ['^GSPC']
    #print(port_list)
    port_list.append(ticker)
    #print(port_list)
    mult_df = merge_df_by_column_name('daily_return', '2018-01-02','2021--09-10',*port_list)
    #print(mult_df)
    cov = mult_df.cov()*252
    #print(cov)
    cov_vs_market = cov.iloc[0,1]
    #print(cov_vs_market)
    sp_var = mult_df['^GSPC'].var()*252
    beta = cov_vs_market/sp_var
    return beta

In [13]:
print('AMD Beta :',find_beta('AMD'))

AMD Beta : 1.4089953431646731


In [34]:
def get_prices_on_date(stocks_df, date):
    #print(stocks_df)
    return stocks_df.loc[pd.DatetimeIndex([date])]['Adj Close'].item()#item will give you just values